# High res drone imagery with TorchGeo

This notebook demonstrates how to use the `OpenAerialMap` dataset in TorchGeo to train a instance segmentation model for building detection.

**Features showcased:**
1. **Catalog Search:** Querying available imagery using the new `search=True` mode.
2. **Data Download:** Downloading specific high-resolution drone imagery.
3. **Intersection Dataset:** combining raster (OAM) and vector (OSM) data.


The system combines a hierarchical vision transformer backbone with an instance segmentation head to predict individual building footprints from aerial imagery. The architecture is implemented in the `Mask2FormerModule` class and consists of three main components:

1. **Swin Transformer V2 Backbone**: Extracts multi-scale features using window-based self-attention
2. **Mask2Former Decoder**: Generates instance-level predictions via learnable queries
3. **Multi-Component Loss**: Optimizes for classification, overlap, and geometric precision


In [1]:
import os
from pathlib import Path
from torchgeo.datasets import OpenStreetMap, OpenAerialMap

## 1. Setup Region of Interest

We define a bounding box for **Banepa Municipality, Nepal**. This area has good drone coverage in OpenAerialMap.

![image.png](docs/train_val.png)

In [2]:
WORK_DIR = Path('data/banepa')
os.makedirs(WORK_DIR, exist_ok=True)

TRAIN_BBOX = [
  85.51176609880189,
  27.625518932561256,
  85.52513148143508,
  27.63551883131749
]

TEST_BBOX = [
  85.53039880381334,
  27.62456651360527,
  85.53606027956683,
  27.629042810653335
]

VAL_BBOX = [
  85.51883176039746,
  27.63560,
  85.52308324197179,
  27.63833629629815
]


## 2. Browse Available Imagery

We use `search=True` to query the catalog without downloading files. This allows us to inspect metadata and choose the best image.

In [3]:
print('Searching for available images...')
browser = OpenAerialMap(paths=WORK_DIR, bbox=TRAIN_BBOX, search=True, max_items=5)

if browser.search_results is not None:
    display(browser.search_results.head())

Searching for available images...
Found 4 available images

Use .search_results to view.



,ID,Date,Platform,Provider,GSD,Title
0,6708341a4a0ab60001b0b94d,2024-10-10T20:07:54.049+00:00,satellite,Maxar,0.345278,Maxar 105001003E1F0500 Nepal
1,67084ad84a0ab60001b0b95b,2024-10-10T21:44:56.060+00:00,satellite,Maxar,0.345283,Maxar 1040010095519A00 Nepal
2,62d86c65d8499800053796c4,2022-04-15T19:00:00Z,uav,Geomatics Engineering Society,0.031627,UAV Images of Banepa Municipality
3,59e62b743d6412ef72209204,,satellite,Digital Globe,0.400000,15APR27052125-S3DS_R14C5-054335918020_01_P001.TIF


## 3. Initialize Datasets

We select a specific Image ID from the search results above and download it.
We also download OpenStreetMap building footprints for the same area.

In [4]:
all_datasets = [TRAIN_BBOX, VAL_BBOX, TEST_BBOX ]

In [5]:
# Selected Image ID (from search results above)
IMAGE_ID = '62d86c65d8499800053796c4'
ZOOM_LEVEL = 19
CHIP_SIZE_PX = 512

print('Initializing OpenAerialMap (Image Layer)...')


folder = ['train', 'val', 'test']

i = 0
for bbox_d in all_datasets:
    oam_dataset = OpenAerialMap(
        paths=os.path.join(WORK_DIR,folder[i],'source'),
        bbox=bbox_d,
        zoom=ZOOM_LEVEL,
        download=True,
        image_id=IMAGE_ID,
        tile_size=CHIP_SIZE_PX,
    )
    
    
    print('Initializing OpenStreetMap (Mask Layer)...')
    OSM_CLASSES = [{'name': 'building', 'selector': [{'building': '*'}]}]

    osm_dataset = OpenStreetMap(
        paths=os.path.join(WORK_DIR, folder[i], 'labels'), bbox=bbox_d, classes=OSM_CLASSES, download=True
    )
    
    i += 1

Initializing OpenAerialMap (Image Layer)...
Using OpenAerialMap image: UAV Images of Banepa Municipality
  ID: 62d86c65d8499800053796c4
  Collection: openaerialmap
  Date: 2022-04-15T19:00:00Z
  Platform: uav
  Provider: Geomatics Engineering Society
  GSD: 0.0316273953754
  License: CC-BY-4.0
Starting download of 378 tiles...
Finished downloading 378 tiles.
Download complete.
Initializing OpenStreetMap (Mask Layer)...
Using OpenAerialMap image: UAV Images of Banepa Municipality
  ID: 62d86c65d8499800053796c4
  Collection: openaerialmap
  Date: 2022-04-15T19:00:00Z
  Platform: uav
  Provider: Geomatics Engineering Society
  GSD: 0.0316273953754
  License: CC-BY-4.0
Starting download of 40 tiles...
Finished downloading 40 tiles.
Download complete.
Initializing OpenStreetMap (Mask Layer)...
Using OpenAerialMap image: UAV Images of Banepa Municipality
  ID: 62d86c65d8499800053796c4
  Collection: openaerialmap
  Date: 2022-04-15T19:00:00Z
  Platform: uav
  Provider: Geomatics Engineering S

## 4. Intersection Dataset & Sampler

We use the `&` operator to create an IntersectionDataset. This ensures every sample contains both imagery and a corresponding mask. this is done inside the package , lets initialize the configuration with dataloaders

In [6]:
from src.config import Config


cfg = Config()

cfg.data_root = WORK_DIR
cfg.output_dir = 'outputs/banepa_experiment'
cfg.train_regions = ['train']
cfg.val_regions = ['val']
cfg.test_regions = ['test']
cfg.wandb_run_name='notebook_run_banepa'
cfg.use_wandb = False 


cfg.epochs = 50

cfg.boundary_loss_weight = 5.0

print(cfg)

Config(data_root=PosixPath('data/banepa'), output_dir='outputs/banepa_experiment', seed=64, train_regions=['train'], val_regions=['val'], test_regions=['test'], val_split=0.2, pretrained_model='facebook/mask2former-swin-base-IN21k-coco-instance', epochs=50, batch_size=8, dice_weight=5.0, mask_weight=5.0, class_weight=5.0, boundary_loss_weight=5.0, learning_rate=1e-05, weight_decay=0.0001, early_stopping_patience=10, num_workers=31, use_wandb=False, wandb_project='building-seg-mask2former', wandb_run_name='notebook_run_banepa')


## 5. Model Setup

The `Mask2FormerModule` class wraps the pretrained `Mask2FormerForUniversalSegmentation` model from Hugging Face Transformers. Model configuration occurs in the `__init__` method:

| Configuration Parameter | Value | Purpose |
|------------------------|-------|---------|
| `pretrained_model` | `facebook/mask2former-swin-base-IN21k-coco-instance` | Base model with ImageNet-22K pretrained backbone |
| `num_labels` | 2 | Binary classification: background (0), building (1) |
| `ignore_index` | 255 | Label value to ignore in loss computation (empty masks) |
| `class_weight` | 5.0 (default) | Weight for classification loss component |
| `dice_weight` | 5.0 (default) | Weight for Dice loss component |
| `mask_weight` | 5.0 (default) | Weight for mask loss component |

The model is initialized from pretrained weights with `ignore_mismatched_sizes=True` to accommodate the change from COCO's 80 classes to binary classification.

![arch](docs/transformer_arc.png)

In [7]:
from src.utils import set_seed
from src.stage1_foundation import Mask2FormerModule
from src.stage1_foundation import OAMDataModule
set_seed(cfg.seed)

model = Mask2FormerModule(cfg)
datamodule = OAMDataModule(cfg)


/home/krschap/code/foss/high-res-building-seg-swinv2-mask2former/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/krschap/code/foss/high-res-building-seg-swinv2-mask2former/.venv/lib/python3.13/site-packages/transformers/image_processing_base.py:417: UserWarning: The following named arguments are not valid for `Mask2FormerImageProcessor.__init__` and were ignored: '_max_size', 'reduce_labels'
  image_processor = cls(**image_processor_dict)
Some weights of Mask2FormerForUniversalSegmentation were not initialized from the model checkpoint at facebook/mask2former-swin-base-IN21k-coco-instance and are newly initialized because the shapes did not match:
- class_predictor.weight: found shape torch.Size([81, 256]) in the checkpoint and torch.Size([3, 256]) in the model instantiated
- class_predictor

## 6. Training Loop

The model uses the AdamW optimizer with cosine annealing learning rate schedule, configured in the `configure_optimizers()` method:

| Parameter | Default Value | Description |
|-----------|---------------|-------------|
| Optimizer | AdamW | Weight decay variant of Adam |
| Learning Rate | 1e-5 | Initial learning rate from `cfg.learning_rate` |
| Weight Decay | 1e-4 | L2 regularization penalty from `cfg.weight_decay` |
| Scheduler | CosineAnnealingLR | Gradually reduces LR to 0 over training |
| T_max | `cfg.epochs` | Number of epochs for full cosine cycle |

The cosine annealing schedule provides:
- Smooth learning rate decay from initial value to near-zero
- Better convergence properties than step decay
- No need for manual LR adjustment during training

The scheduler updates once per epoch (`interval="epoch"`, `frequency=1`) with strict enforcement of the schedule.

In [ ]:
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
import pytorch_lightning as pl

callbacks = [
    EarlyStopping(
        monitor="val_loss",
        patience=cfg.early_stopping_patience,
        mode="min",
    ),
    ModelCheckpoint(
        dirpath=cfg.output_dir,
        filename="best",
        monitor="val_loss",
        mode="min",
        save_top_k=1,
    ),
]

logger = (
    WandbLogger(project=cfg.wandb_project, name=cfg.wandb_run_name)
    if cfg.use_wandb
    else None
)

trainer = pl.Trainer(
    max_epochs=cfg.epochs,
    accelerator="auto",
    devices="auto",
    callbacks=callbacks,
    logger=logger,
    precision="16-mixed",
    default_root_dir=cfg.output_dir,
)

trainer.fit(model, datamodule)


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


## 7. Test & Visualize Instances

In [ ]:
trainer.test(model, datamodule, ckpt_path="best", weights_only=False)

## Metrics

**Instance-Level (AP/mAP)** - Evaluates each building separately:
- `map` - mAP @ IoU 0.50:0.95 (strict, best for boundaries)
- `map_50` - mAP @ IoU 0.50 (detection quality)

**Pixel-Level (Binary)** - Overall coverage:
- `iou` - Intersection over Union
- `f1` - F1 score
- `acc` - Accuracy

For building boundaries & separation, prioritize **`map`** and **`map_50`**

In [ ]:
datamodule.setup()
batch = next(iter(datamodule.test_dataloader()))
model.visualize_batch(batch, num_samples=5, save_path="test_results.png")